# Cloud Front with API Gateway Origin

In [ ]:
# SDK Imports
import boto3

cformation_east = boto3.client('cloudformation', region_name='us-east-1')
cformation_west = boto3.client('cloudformation', region_name='us-west-2')

gw_east = boto3.client('apigateway')

In [ ]:
service = 'serverless-rest-api-with-dynamodb'
stage = 'pp1'

In [ ]:
def get_stack_name(service, stage):
    return '{}-{}'.format(service,stage)

In [ ]:
def get_endpoint(cf_client, stack_name):
    response = cf_client.describe_stacks(
        StackName=stack_name
    )
    
    outputs = response['Stacks'][0]['Outputs']
    endpoint =  [d for d in outputs if d['OutputKey'] == 'ServiceEndpoint'][0]['OutputValue']
    return endpoint

In [ ]:
east_endpoint = get_endpoint(cformation_east, get_stack_name(service, stage))
print east_endpoint

In [ ]:
# Create a key and add it to the usage plan?
# - create_api_key - need key id output
# - you can get the usage plan id and the api id via get_usage_plan and matching the plan with same name
#   as the stack
# - create_usage_plan_key associates the key to the plan: inputs are plan id, key id

In [ ]:
response = gw_east.get_usage_plans()
plans = response['items']
stack_name = get_stack_name(service, stage)
plan =  [d for d in plans if d['name'] == stack_name][0]
plan_id = plan['id']
print plan_id
api_stage = [d for d in plan['apiStages'] if d['stage'] == stage][0]
api_id = api_stage['apiId']
print api_id

In [ ]:
import uuid
id = str(uuid.uuid4())
#id = id.replace("-","")
#id = id[0:20]
print id
#print len(id)

In [ ]:
create_key_response = gw_east.create_api_key(
    name='xregion_key',
    enabled=True,
    generateDistinctId=True,
    value=id
)

print create_key_response

In [ ]:
key_id = create_key_response['id']
response = gw_east.delete_api_key(
    apiKey=key_id
)

print response